# Part 3: Launch a Grid Network Locally

In this tutorial, you'll learn how to deploy a grid network into a local machine and then interact with it using PySyft.

_WARNING: Grid nodes publish datasets online and are for EXPERIMENTAL use only. Deploy nodes at your own risk. Do not use OpenGrid with any data/models you wish to keep private._

In order to run an grid network locally you will need to run two different apps: a grid gateway and one or more grid workers. In this tutorial we will use the websocket app available [here](https://github.com/OpenMined/PyGrid/tree/dev/app/websocket) to start the grid workers.


## Starting the Grid Gateway


### Step 1: Download the repository

```bash
git clone https://github.com/OpenMined/PyGrid/
```


### Step 2: Download dependencies

You'll need to have the app dependencies installed. We recommend setting up an independent [conda environment](https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/environments.html) to avoid problems with library versions.

You can install the dependencies by running:

```bash
cd PyGrid/gateway/
pip install -r requirements.txt
```

### Step 3: Make grid importable

Install grid as a python package

```bash
cd PyGrid
python setup.py install (or python setup.py develop)
```

### Step 4: Start gateway app

Then to start the app just run the `gateway.py` script. The `--start_local_db` automatically starts a local database so you don't have to configure one yourself.

```bash
python gateway.py --start_local_db --port=<port_number>
```

This will start the app on address: `http://0.0.0.0/<port_number>`.

To check what other arguments you can use when running this app, run:

```bash
python gateway.py --help
```

Let's start a grid gateway on port `5000`

```bash
python gateway.py --port=5000
```

Great, so if your app started successfully the script should still be running.

## Starting the Grid Worker App

### Step 5: Starting the Grid Worker app

This is the same procedure already described at Part 1. But we add a new argument when starting the app called `--gateway_url` this should equal to the address used by the grid network here it's "http://localhost:5000"

Let's start two workers:

* bob on port `3000`
* alice on port `3001`

```bash
python websocket_app.py --db_url=redis:///redis:6379 --id=bob --port=3000 --gateway_url=http://localhost:5000
```

```bash
python websocket_app.py --db_url=redis:///redis:6379 --id=alice --port=3001 --gateway_url=http://localhost:5000
```

We should always start the workers after starting the grid gateway!!

Great, so if your app started successfully the script should still be running.


### Step 6: Start communication with the Grid Gateway and workers

Let's start communication with the Gateway and the workers.

In [1]:
# General dependencies
import torch as th
import syft as sy
import grid as gr

hook = sy.TorchHook(th)

In [2]:
gateway = gr.GridNetwork("http://localhost:5000")

In [3]:
# WARNING: We should use the same id and port as the one used to start the app!!!
bob = gr.WebsocketGridClient(hook, id="bob", address="http://localhost:3000")
# If you don't connect to the worker you can't send messages to it
bob.connect()

# WARNING: We should use the same id and port as the one used to start the app!!!
alice = gr.WebsocketGridClient(hook, id="alice", address="http://localhost:3001")
# If you don't connect to the worker you can't send messages to it
alice.connect()

### Step 7: Use PySyft Like Normal

Now you can simply use the worker you created like you would any other normal PySyft worker. For more on how PySyft works, please see the PySyft tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials

In [4]:
x = th.tensor([1,2,3,4]).send(bob)
x

(Wrapper)>[PointerTensor | me:29680419628 -> bob:81752950542]

In [5]:
y = x + x
y

(Wrapper)>[PointerTensor | me:18859963688 -> bob:42466099899]

In [6]:
y.get()

tensor([2, 4, 6, 8])

### Step 7: Perform operations on Grid Network

So far we haven't done anything different, but here is the magic: we can interact with the network to query general information about it.

In [7]:
x = th.tensor([1, 2, 3, 4, 5]).tag("#tensor").send(bob)

We can search for a tensor in the entire network, and get pointers to all tensors.

In [8]:
gateway.search("#tensor")

[[(Wrapper)>[PointerTensor | me:1797921498 -> bob:1664766854]]]

In [9]:
y = th.tensor([1, 2, 3, 4, 5]).tag("#tensor").send(alice)

In [10]:
gateway.search("#tensor")

[[(Wrapper)>[PointerTensor | me:40497032036 -> bob:1664766854]],
 [(Wrapper)>[PointerTensor | me:75678714045 -> alice:96959474372]]]